<a href="https://colab.research.google.com/github/Kandeo/Moringa-Data-Science/blob/main/Christine_Kandeo_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

In this week's independent project, you will be working as a data scientist working for an electric car-sharing service company. You have been tasked to process stations data to understand electric car usage over time by solving for the following research question;

# Research Question:

1. Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

Bonus Questions:

2. What is the most popular hour for returning cars?
3. What station is the most popular?
      Overall?
      At the most popular picking hour?
4. What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?
      Overall?
      At the most popular picking hour?
5. Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars? 

Dataset:

 [http://bit.ly/autolib_dataset (Links to an external site.)]   
 The dataset contains data collected for a period of 9 days.

Importing Libraries

In [67]:
#Importing the Pandas Library:
import pandas as pd 

#Importing the Numpy Library:
import numpy as np


# Loading Our Dataset

In [68]:
#Reading the data from our CSV File:

df = pd.read_csv('Autolib_dataset (2).csv')

#Previewing the first 5 rows:
df.head(5)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


Accessing Information About The Dataset

In [69]:
#To get information about our dataset:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

# Data Cleaning

Validity

In [70]:
#Removing any irrelevant observation:
#Dropping columns: Address, Displayed Comments,ID, Geo Point, Public name and Scheduled at  
#The columns are not important in our analysis

df.drop(df.columns[[0, 8, 9, 11, 15]], axis = 1, inplace = True)
df.head(4)

,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Kind,Postal code,Public name,Rental status,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,0,0,0,0,0,nonexistent,Paris,STATION,75015,Paris/Suffren/2,operational,2,station,ok,nonexistent,2018,4,8,11,43
1,6,6,0,0,0,operational,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,station,ok,nonexistent,2018,4,6,7,24
2,3,3,0,2,0,operational,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,station,ok,nonexistent,2018,4,3,20,14
3,3,3,1,0,1,operational,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,station,ok,nonexistent,2018,4,4,4,37


In [71]:
#Removing any white spaces from Our String values and adding _:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('.', '')
df.head(4)

,Cars,Bluecar_counter,Utilib_counter,Utilib_14_counter,Charge_Slots,Charging_Status,City,Kind,Postal_code,Public_name,Rental_status,Slots,Station_type,Status,Subscription_status,year,month,day,hour,minute
0,0,0,0,0,0,nonexistent,Paris,STATION,75015,Paris/Suffren/2,operational,2,station,ok,nonexistent,2018,4,8,11,43
1,6,6,0,0,0,operational,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,station,ok,nonexistent,2018,4,6,7,24
2,3,3,0,2,0,operational,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,station,ok,nonexistent,2018,4,3,20,14
3,3,3,1,0,1,operational,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,station,ok,nonexistent,2018,4,4,4,37


Completeness

In [72]:
#Checking and Counting any missing values:
df.isnull().values.any().sum()

0

Consistency

In [73]:
#Checking for any Duplicates:
df.duplicated().sum()

0

In [74]:
#Checking on the column Cars and Bluecar counter for redundancy as per the comments:

def Check(df):
   if df['Cars']== df['Bluecar_counter']:
      return "True"
   else:
      return "False" 

df['result'] = df.apply(Check, axis=1)
df

#Both columns have similar values as per result column

,Cars,Bluecar_counter,Utilib_counter,Utilib_14_counter,Charge_Slots,Charging_Status,City,Kind,Postal_code,Public_name,Rental_status,Slots,Station_type,Status,Subscription_status,year,month,day,hour,minute,result
0,0,0,0,0,0,nonexistent,Paris,STATION,75015,Paris/Suffren/2,operational,2,station,ok,nonexistent,2018,4,8,11,43,True
1,6,6,0,0,0,operational,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,station,ok,nonexistent,2018,4,6,7,24,True
2,3,3,0,2,0,operational,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,station,ok,nonexistent,2018,4,3,20,14,True
3,3,3,1,0,1,operational,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,station,ok,nonexistent,2018,4,4,4,37,True
4,3,3,0,0,0,nonexistent,Paris,PARKING,75017,Paris/Porte de Champerret/6,operational,3,station,ok,nonexistent,2018,4,8,17,23,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2,2,0,0,0,nonexistent,Colombes,STATION,92700,Colombes/MÃ©nelotte/8,operational,3,station,ok,nonexistent,2018,4,6,11,26,True
4996,4,4,0,0,1,operational,Paris,STATION,75015,Paris/Dantzig/37,operational,2,station,ok,nonexistent,2018,4,4,16,56,True
4997,1,1,0,0,1,operational,Paris,STATION,75007,Paris/Bac/142,operational,4,station,ok,nonexistent,2018,4,1,7,1,True
4998,2,2,0,0,0,nonexistent,Fontenay-Sous-Bois,STATION,94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,3,station,ok,nonexistent,2018,4,4,17,27,True


In [75]:
#Dropping the Cars column:
df.drop('Cars',inplace= True, axis=1)
df.head(4)

,Bluecar_counter,Utilib_counter,Utilib_14_counter,Charge_Slots,Charging_Status,City,Kind,Postal_code,Public_name,Rental_status,Slots,Station_type,Status,Subscription_status,year,month,day,hour,minute,result
0,0,0,0,0,nonexistent,Paris,STATION,75015,Paris/Suffren/2,operational,2,station,ok,nonexistent,2018,4,8,11,43,True
1,6,0,0,0,operational,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,station,ok,nonexistent,2018,4,6,7,24,True
2,3,0,2,0,operational,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,station,ok,nonexistent,2018,4,3,20,14,True
3,3,1,0,1,operational,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,station,ok,nonexistent,2018,4,4,4,37,True


In [76]:
#Checking the shape of out dataset:
df.shape


(5000, 20)

Uniformity

In [77]:
#To view the datatypes in our set:
df.dtypes

Bluecar_counter         int64
Utilib_counter          int64
Utilib_14_counter       int64
Charge_Slots            int64
Charging_Status        object
City                   object
Kind                   object
Postal_code             int64
Public_name            object
Rental_status          object
Slots                   int64
Station_type           object
Status                 object
Subscription_status    object
year                    int64
month                   int64
day                     int64
hour                    int64
minute                  int64
result                 object
dtype: object

In [78]:
#Changing the data type of year, month, day, hour and minute Columns to datetime:
df['Date_Time']=pd.to_datetime(df[['year','month','day', 'hour', 'minute']])
df.head(4)

#Dropping the year, month, day, hour and minute Columns:
df.drop(df.columns[[13, 14, 15, 16, 17]], axis = 1, inplace = True)
df.head(4)

,Bluecar_counter,Utilib_counter,Utilib_14_counter,Charge_Slots,Charging_Status,City,Kind,Postal_code,Public_name,Rental_status,Slots,Station_type,Status,minute,result,Date_Time
0,0,0,0,0,nonexistent,Paris,STATION,75015,Paris/Suffren/2,operational,2,station,ok,43,True,2018-04-08 11:43:00
1,6,0,0,0,operational,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,0,station,ok,24,True,2018-04-06 07:24:00
2,3,0,2,0,operational,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,1,station,ok,14,True,2018-04-03 20:14:00
3,3,1,0,1,operational,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,2,station,ok,37,True,2018-04-04 04:37:00


Exporting Cleaned dataset

In [79]:
#To export our cleaned dataset:
df.to_csv('Autolib_dataset(2).csv')

# Data Analysis

To Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

In [107]:
# As per the hint given:
#Let   - number be picked
#      + number be Returned
#      0 for nothing happened
#Adding Blue1, Utilib1 and Utilib2 to represent this Activity for the 3:      

df['Blue1'] = df.Bluecar_counter.diff()
df['Utilib1'] = df.Utilib_counter.diff()
df['Utilib2'] = df.Utilib_14_counter.diff()
df.head(4)

#Most popular hour for Picking:
#Sorting by City=Paris, and using Blue1 column:

df[(df.City == 'Paris') & (df.Blue1 < 0)].groupby('Date_Time')['Date_Time'].count().sort_values(ascending = False).head(1)

Date_Time
2018-04-08 03:45:00    3
Name: Date_Time, dtype: int64

The most popular hour for returning cars

In [81]:
#Most popular hour for returning:
#In returning, the values are greater than 0

df[df.Blue1 > 0].groupby('Date_Time')['Date_Time'].count().sort_values(ascending = False).head(1)

Date_Time
2018-04-04 18:20:00    3
Name: Date_Time, dtype: int64

What station is the most popular, Overall and At the most popular picking hour?

In [91]:
#Overall
df[df['Kind'] == 'STATION'].groupby('Public_name')['Kind'].count().sort_values(ascending=False).head(1)

Public_name
Paris/Porte de Montrouge/8    13
Name: Kind, dtype: int64

In [92]:
#At the most popular hour
#df[df['Kind'] == 'STATION'].groupby('Public_name')['Kind'].count().sort_values(ascending=False).head(1)

df[(df.Kind == 'STATION') & (df.Blue1 < 0)].groupby('Public_name')['Kind'].count().sort_values(ascending=False).head(1)


Public_name
Paris/Bernardins/23    7
Name: Kind, dtype: int64

Most popular Postal Code for piking blue cars, Overall and At the most popular picking hour?

In [105]:
#Overall
df[df.Blue1 < 0].groupby('Postal_code').count().sort_values(by = 'Blue1', ascending=False).head(1)

,Bluecar_counter,Utilib_counter,Utilib_14_counter,Charge_Slots,Charging_Status,City,Kind,Public_name,Rental_status,Slots,Station_type,Status,minute,result,Date_Time,Blue1,Utilib1,Utilib2
Postal_code,,,,,,,,,,,,,,,,,,
75015,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116


Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars

In [109]:
#To check for unique values while picking and returning:

df.Utilib1.unique()

df.Utilib2.unique()

#The data contains Nan Values 

array([nan,  0.,  2., -2.,  1., -1.,  3., -3.])